# 1. 单局 PK 可视化区域

In [8]:
from gomoku.player import ZeroMCTSPlayer, WrongZeroMCTSPlayer, IneffectiveZeroMCTSPlayer, play_one_game
from gomoku.gomoku_env import GomokuEnvSimple, GomokuEnv
from gomoku.policy import ZeroPolicy

In [13]:
from gomoku.player import IneffectiveZeroMCTSPlayer
import torch

board_size = 7

policy = ZeroPolicy(board_size, num_blocks=2)
# policy.load_state_dict(torch.load('continue_model/policy_step_940000.pth'))
# policy.load_state_dict(torch.load('../models/gomoku_zero_9_pre5/policy_step_800000.pth'))
policy.load_state_dict(torch.load('../models/gomoku_zero_9_lab_1/policy_step_80000.pth'))
player1 = ZeroMCTSPlayer(policy)
player2 = ZeroMCTSPlayer(policy)

game = GomokuEnv(board_size)

info = play_one_game(player1, player2, game=game,board_size=board_size, render=True, eager=True, itermax=200)

   0  1  2  3  4  5  6
 0 .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  . 
 3 .  .  .  X  .  .  . 
 4 .  .  .  .  .  .  . 
 5 .  .  .  .  .  .  . 
 6 .  .  .  .  .  .  . 

   0  1  2  3  4  5  6
 0 .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  . 
 3 .  .  .  X  .  .  . 
 4 .  .  .  O  .  .  . 
 5 .  .  .  .  .  .  . 
 6 .  .  .  .  .  .  . 

   0  1  2  3  4  5  6
 0 .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  . 
 3 .  .  .  X  X  .  . 
 4 .  .  .  O  .  .  . 
 5 .  .  .  .  .  .  . 
 6 .  .  .  .  .  .  . 

   0  1  2  3  4  5  6
 0 .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  . 
 3 .  .  O  X  X  .  . 
 4 .  .  .  O  .  .  . 
 5 .  .  .  .  .  .  . 
 6 .  .  .  .  .  .  . 

   0  1  2  3  4  5  6
 0 .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  . 
 3 .  .  O  X  X  .  . 
 4 .  .  .  O  X  .  . 
 5 .  .  .  .  .  .  . 
 6 .  .  .  .  .  .  . 

   0  1  2  3  4  5  6
 0 .  .  .  .  . 

In [ ]:
import cProfile
import torch
import pstats
import io

# --- 开始性能分析 ---

# 1. 创建一个 Profiler 对象
profiler = cProfile.Profile()

# 2. 启用 Profiler 并运行你的函数
profiler.enable()
policy = ZeroPolicy(board_size=15)
policy.load_state_dict(torch.load('models/gomoku_zero_15_continue/policy_step_6000.pth'))
player1 = ZeroMCTSPlayer(policy, itermax=200, device='cpu', eager=True)
player2 = ZeroMCTSPlayer(policy, itermax=200, device='cpu', eager=True)

game = GomokuEnv(15)

info = play_one_game(player1, player2, board_size=15, render=True)

profiler.disable()

# 3. 创建一个 IO 流来捕获分析结果
s = io.StringIO()

# 4. 创建 pstats.Stats 对象来格式化和排序结果
#    sort_stats() 的参数是排序依据，'cumulative' 是按累计耗时排序
stats = pstats.Stats(profiler, stream=s).sort_stats('cumulative')

# 5. 打印分析报告
stats.print_stats()

# 6. (可选) 只打印前 10 个最耗时的函数
# stats.print_stats(10)

print(s.getvalue())


# 2. ARENA 

In [2]:
from gomoku.player import arena_parallel
from gomoku.player import ZeroMCTSPlayer, WrongZeroMCTSPlayer, IneffectiveZeroMCTSPlayer, play_one_game
from gomoku.gomoku_env import GomokuEnvSimple
from gomoku.policy import ZeroPolicy

/home/zhangpeng.pada/GomokuZero/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-08 15:35:20,991	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [16]:
policy1 = ZeroPolicy(board_size)
policy2 = ZeroPolicy(board_size)
policy1.load_state_dict(torch.load('../models/gomoku_zero_9_lab_5/policy_step_100000.pth'))
policy2.load_state_dict(torch.load('../models/gomoku_zero_9_lab_1/policy_step_100000.pth')) 

r = arena_parallel(
    policy1,
    policy2, 
    board_size=7,
    num_cpus=8,
    games=300,
    itermax=100,
)

Starting parallel arena with 300 games on 8 CPUs...
Arena finished!
Player 1 wins: 133 (44.33%)
Player 2 wins: 142 (47.33%)
Draws: 25 (8.33%)
函数 'arena_parallel' 执行耗时: 64.3855 秒


In [ ]:
from worker import gather_selfplay_games


gather_selfplay_games(
    policy=ZeroPolicy(board_size=9),
    device='cpu',
    board_size=9,
    num_workers=10,
    games_per_worker=10,
    itermax=100,
)

In [ ]:
from player import self_play


_ = self_play(
    policy=ZeroPolicy(board_size=9),
    device='cpu',
    board_size=9,
    itermax=100,
)

In [ ]:


_ = play_one_game(
    player1=ZeroMCTSPlayer(ZeroPolicy(board_size=9)),
    player2=ZeroMCTSPlayer(ZeroPolicy(board_size=9)),
    board_size=9,
    # render=True,
    game=GomokuEnvSimple(board_size=9),
    itermax=100
)

# 3. 评价棋局

In [4]:
from gomoku.player import ZeroMCTSPlayer, WrongZeroMCTSPlayer, IneffectiveZeroMCTSPlayer, play_one_game, self_play
from gomoku.gomoku_env import GomokuEnvSimple
from gomoku.policy import ZeroPolicy
import torch

/home/zhangpeng.pada/GomokuZero/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-07 16:22:06,128	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [14]:
board_size = 9

policy = ZeroPolicy(board_size)
policy.load_state_dict(torch.load('../models/gomoku_zero_9_pre5/policy_step_710000.pth'))

device='cpu'
player1 = ZeroMCTSPlayer(policy, device=device) 
player2 = ZeroMCTSPlayer(policy, device=device)

env = GomokuEnvSimple(board_size)

winner, infos = play_one_game(
    player1, player2, board_size=board_size, game=env,
    itermax=200, eager=False
)
# print(f"Game over! Winner: {winner}")
# return infos

函数 'play_one_game' 执行耗时: 7.5808 秒


In [15]:
# infos = self_play(policy, 'cpu', board_size=board_size, itermax=400)


In [16]:
def render(states, index):
    import numpy as np
    states_tensor = torch.from_numpy(np.array(infos['states'], dtype=np.float32))
    probs, winrate = policy(states_tensor)

    symbols = {0: '.', 1: 'X', 2: 'O'}
    board_str = "  " + " ".join([f"{i:2d}" for i in range(9)]) + "\n"
    
    for i in range(9):
        cells = []
        for j in range(9):
            if states[index][ 0, i, j] ==  1:  
                cell = 1
            elif states[index][ 1, i, j] == 1:
                cell = 2
            else:
                cell = 0
            cells.append(cell)
        board_str += f"{i:2d} " + " ".join([f"{symbols[cell]} " for cell in cells]) + "\n"
    print(board_str)
    print(winrate[index])
    values, indices = torch.topk(torch.softmax(probs, dim=-1)[index], 10)

    def action_2_index(action):
        return action // 9, action % 9

    for i  in range(len(indices)):
        a, b = action_2_index(indices[i].item())
        print(f"[{a:2d},{b:2d}]:{values[i].item():.4f}")


In [19]:
render(infos['states'], 2)

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  .  .  . 
 3 .  .  .  .  .  .  O  .  . 
 4 .  .  .  X  .  .  .  .  . 
 5 .  .  .  .  .  .  .  .  . 
 6 .  .  .  .  .  .  .  .  . 
 7 .  .  .  .  .  .  .  .  . 
 8 .  .  .  .  .  .  .  .  . 

tensor([0.7832], grad_fn=<SelectBackward0>)
[ 4, 4]:0.9994
[ 5, 4]:0.0002
[ 3, 4]:0.0001
[ 3, 5]:0.0001
[ 3, 3]:0.0001
[ 5, 3]:0.0000
[ 5, 5]:0.0000
[ 4, 2]:0.0000
[ 4, 6]:0.0000
[ 2, 3]:0.0000


In [ ]:
render(info[1]['states'], -2)

In [ ]:
def action_2_index(action):
    return action // 9, action % 9

probs = torch.tensor(info[1]['probs'])
values, indices = probs[-2].topk(10)

for i  in range(len(indices)):
    a, b = action_2_index(indices[i].item())
    print(f"[{a:2d},{b:2d}]:{values[i].item():.4f}")


In [1]:
# 高手只需运行这个脚本就能让他们的 AI 和你的对战
import websocket

ws = websocket.WebSocket()
ws.connect("ws://localhost:8000/ws/play")
ws.send('{"type": "handshake", "bot_name": "DeepSeeker_V1"}')
# ... 循环接收棋盘，计算，发送落子

TypeError: WebSocket.__init__() missing 3 required positional arguments: 'environ', 'socket', and 'rfile'

In [5]:
import torch
import numpy as np

def generate_benchmark_dataset():
    """
    生成 50 个合法的五子棋残局。
    修正点：
    1. 增加干扰子(D)以平衡黑白棋子数量，确保局面合法 (Count(Black) == Count(White) or +1)。
    2. 明确指定 last_move_coord，且必须对应棋盘上的 O。
    """
    dataset_X = []
    dataset_y = []
    
    board_size = 9 # 使用 9x9 进行测试
    
    def parse_board(pattern, target_coords, last_move_coord):
        # 初始化通道
        self_plane = np.zeros((board_size, board_size), dtype=np.float32) # 我方 (X)
        oppo_plane = np.zeros((board_size, board_size), dtype=np.float32) # 对手 (O)
        last_plane = np.zeros((board_size, board_size), dtype=np.float32) # 对手上一手
        target_plane = np.zeros((board_size, board_size), dtype=np.float32)

        rows = pattern.strip().split('\n')
        # 去除每行的首尾空格
        rows = [r.strip() for r in rows]
        
        for r, row_str in enumerate(rows):
            for c, char in enumerate(row_str):
                if char == 'X': # 我方棋子
                    self_plane[r, c] = 1.0
                elif char == 'O': # 对手棋子（关键子）
                    oppo_plane[r, c] = 1.0
                elif char == 'D': # 对手棋子（干扰子，用来平衡数量的）
                    oppo_plane[r, c] = 1.0

        # 设置 Last Move (必须是对手下的)
        if last_move_coord:
            if oppo_plane[last_move_coord[0], last_move_coord[1]] != 1.0:
                print(f"Warning: Last move {last_move_coord} is not on an Opponent stone!")
            last_plane[last_move_coord[0], last_move_coord[1]] = 1.0

        # 设置目标概率
        prob = 1.0 / len(target_coords)
        for tr, tc in target_coords:
            target_plane[tr, tc] = prob
            
        # 校验数量平衡 (假设自己是黑棋先手，轮到自己下，那么当前盘面 黑==白 或 黑==白-1 都是不对的)
        # 轮到 Self 下：
        # 如果 Self 是黑(先)：盘面应该是 Black == White
        # 如果 Self 是白(后)：盘面应该是 Black == White + 1 (即 Opponent 多一个)
        # 为了简化，我们默认生成的局面都是：轮到 Self 走，且双方子数大致平衡
        
        x_tensor = np.stack([self_plane, oppo_plane, last_plane])
        return x_tensor, target_plane

    def augment_and_add(x, y):
        # 旋转翻转扩充数据 (1 -> 8)
        x_trans = np.transpose(x, (1, 2, 0)) 
        for k in range(4):
            x_rot = np.rot90(x_trans, k)
            y_rot = np.rot90(y, k)
            dataset_X.append(np.transpose(x_rot, (2, 0, 1)))
            dataset_y.append(y_rot)
            
            x_flip = np.fliplr(x_rot)
            y_flip = np.fliplr(y_rot)
            dataset_X.append(np.transpose(x_flip, (2, 0, 1)))
            dataset_y.append(y_flip)

    # ================== 题目设计 ==================

    # --- Level 1: 嵌五必杀 (Immediate Win) ---
    # 场景：轮到 X 下。X 有 4 个子，O 也有 4 个子。
    # X 在中间空了一个洞 (0,1,2,3,4 中的 2 是空的)
    # 这是一个平衡局：X:4, O:4 (D是凑数的 O)
    l1_pattern = """
    .........
    DD.......
    .O.......
    .XX.XX...
    .........
    .........
    .........
    .........
    .........
    """ 
    # 分析：X有4个，O有3个(D+D+O)。为了平衡，O必须是4个。
    # 假设对手刚下了 (2, 1) 的 O，试图防守，但没防住下面
    l1_targets = [(3, 3)]
    x, y = parse_board(l1_pattern, l1_targets, last_move_coord=(2, 1))
    augment_and_add(x, y) # +8

    # --- Level 2: 必须堵冲四 (Critical Defense) ---
    # 场景：轮到 X 下。O 连了 4 个 (冲四)，X 必须堵。
    # 盘面：O 有 4 个，X 应该有 3 个或 4 个。
    l2_pattern = """
    .........
    ....O....
    ....O....
    X...O....
    X...O....
    X........
    .........
    .........
    .........
    """
    # 分析：O 竖线 4 个，X 左边 3 个。O 刚下了 (1, 4) 形成冲四。
    # 此时 O=4, X=3。合法（白多黑少，轮到黑下）。
    l2_targets = [(0, 4), (5, 4)] # 两头都要防，或者防一头，这里假设两头均分
    x, y = parse_board(l2_pattern, l2_targets, last_move_coord=(1, 4))
    augment_and_add(x, y) # +8 = 16

    # --- Level 3: 活三进攻 (Active 3) ---
    # 场景：X 形成活三，准备下一步成活四。
    # 盘面：X 有 3 个，O 有 3 个。
    l3_pattern = """
    .........
    D.D......
    .........
    .XXX.....
    ..O......
    .........
    .........
    .........
    .........
    """
    # X: 3个, O: 3个 (D, D, O)。平衡。
    # O 刚下在 (4, 2) 挡路，但没挡住上面。
    l3_targets = [(3, 0), (3, 4)] # 左右延伸
    x, y = parse_board(l3_pattern, l3_targets, last_move_coord=(4, 2))
    augment_and_add(x, y) # +8 = 24

    # --- Level 4: 复杂防守 (VCF 防守) ---
    # 场景：O 形成了 "跳活三" (O O . O)，中间那个点如果不补就是死。
    # 盘面：O: 3个, X: 2个。
    l4_pattern = """
    .........
    .........
    .........
    ..O.O.O..
    ....X....
    ....X....
    .........
    .........
    .........
    """
    # O: 3个, X: 2个。合法（白3黑2，轮到黑）。
    # O 刚下了 (3, 6) 的 O。
    l4_targets = [(3, 3), (3, 5), (3, 7)] # (3,5)是填中间，最优；(3,3)/(3,7)是堵两头
    # 为了简化，我们只要求堵中间
    l4_targets = [(3, 5)] 
    x, y = parse_board(l4_pattern, l4_targets, last_move_coord=(3, 6))
    augment_and_add(x, y) # +8 = 32

    # --- Level 5: 开局 (Opening) ---
    # 场景 A: 空棋盘。黑棋先手。
    # Last Move 是 None (全0)
    l5_a_pattern = """
    .........
    .........
    .........
    .........
    .........
    .........
    .........
    .........
    .........
    """
    l5_a_targets = [(4, 4)]
    # 特殊处理：last_move_coord 设为 None
    x, y = parse_board(l5_a_pattern, l5_a_targets, last_move_coord=None) 
    # 这是一个特殊的 Case，手动处理 last plane 全 0
    x[2] = np.zeros((board_size, board_size)) 
    
    dataset_X.append(x)
    dataset_y.append(y)
    
    # 场景 B: 必应点 (Standard Response)
    # 黑下了天元，白下了直指 (3, 4)。轮到黑下。
    # X:1, O:1。
    l5_b_pattern = """
    .........
    .........
    .........
    ....O....
    ....X....
    .........
    .........
    .........
    .........
    """
    # X: (4,4), O: (3,4)。O 刚下的 (3,4)。
    # 常用点：(5,5), (3,5), (5,3)
    l5_b_targets = [(5, 5), (3, 5), (5, 3)]
    x, y = parse_board(l5_b_pattern, l5_b_targets, last_move_coord=(3, 4))
    augment_and_add(x, y) # +8 = 41

    # 补齐剩余数据 (用 Level 2 的变体 - 仅仅是方向不同)
    l_supp_pattern = """
    .........
    ..X......
    ..X.O....
    ..X.O....
    ....O....
    ....O....
    .........
    .........
    .........
    """
    # O 竖线4个，X 竖线3个。O 刚下了 (5, 4)。
    l_supp_targets = [(2, 4), (6, 4)]
    x, y = parse_board(l_supp_pattern, l_supp_targets, last_move_coord=(5, 4))
    augment_and_add(x, y) # +8 = 49
    
    # 凑第50个，复制一下空棋盘
    dataset_X.append(dataset_X[32])
    dataset_y.append(dataset_y[32])

    X_tensor = torch.tensor(np.array(dataset_X[:50]), dtype=torch.float32)
    y_tensor = torch.tensor(np.array(dataset_y[:50]), dtype=torch.float32)

    return X_tensor, y_tensor

if __name__ == "__main__":
    X, y = generate_benchmark_dataset()
    print("X shape:", X.shape) # 应为 (50, 3, 9, 9)
    print("y shape:", y.shape) # 应为 (50, 9, 9)
    
    # 验证一下数据是否正常
    sample_idx = 10 # 随便取一个
    print("\nSample Board (Channel 0 - Self):")
    print(X[sample_idx][0])
    print("\nSample Board (Channel 1 - Opponent):")
    print(X[sample_idx][1])
    print("\nSample Board (Channel 2 - Last Move):")
    print(X[sample_idx][2])
    print("\nTarget:")
    print(y[sample_idx])

X shape: torch.Size([50, 3, 9, 9])
y shape: torch.Size([50, 9, 9])

Sample Board (Channel 0 - Self):
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0.]])

Sample Board (Channel 1 - Opponent):
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

Sample Board (Channel 2 - Last

In [27]:
import torch.nn.functional as F


def Evaluate(poliy, X, y):
    logits = policy(X)[0]

    probs = F.softmax(logits, dim=-1)
    print(probs)
    y = y.reshape(y.shape[0],-1)
    cse = torch.sum(y * probs, dim=1).sum() # 先在每个样本上求和，再在batch上求平均

    # print(cse)
    return cse

In [7]:
# Evaluate(policy, X, y)

NameError: name 'policy' is not defined

In [24]:
policy = ZeroPolicy(9, num_blocks=2)
# policy.load_state_dict(torch.load('continue_model/policy_step_940000.pth'))
policy.load_state_dict(torch.load('../models/gomoku_zero_9_pre5/policy_step_100000.pth'))
Evaluate(policy, X, y)

tensor(5.5245, grad_fn=<SumBackward0>)

In [2]:
from gomoku.policy import ZeroPolicy
from gomoku.evaluate import Evaluate

policy = ZeroPolicy(7, num_blocks=2)
# policy.load_state_dict(torch.load('continue_model/policy_step_940000.pth'))
# policy.load_state_dict(torch.load('../models/gomoku_zero_9_pre5/policy_step_950000.pth'))
# policy.load_state_dict(torch.load('continue_model/policy_step_940000.pth'))
Evaluate(policy, board_size=7)

tensor(1.0393, grad_fn=<SumBackward0>)

In [42]:
# y[:1]
# X[:1]
# X[-1]
for i in range(50):
    if X[i][0].sum() == 0:
        print(i)
X[32]

32
49


tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         